In [1]:
import sys #check the current environment
print(sys.executable)

D:\Applications\Programming\Anaconda3\envs\snap_esa-env\python.exe


In [2]:
# MODULE                                # DESCRIPTION
import os
from os.path import join                # data access in file manager  
from glob import iglob                  # data access in file manager
import subprocess                       # external calls to system

import esa_snappy                       # SNAP python interface


Modules imported done


In [3]:
# Set output directory
output_dir = r"E:\MA_thesis\Processed_Data\S1_AtkaBay_2022_sigma0_dB"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
### Read Sentinel-1 Products

# Set target folder and extract metadata
product_path = r"F:\Snow_Drift_Thesis\Data\SAR_Atka Bay\S1_2022_Atka"
input_S1_files = sorted(list(iglob(join(product_path, '**', '*S1*.zip'), recursive=True)))

s1_src_all = []  #create a list to store all raw S1 data
file_name = []
file_date =[]

for i, file_path in enumerate(input_S1_files):
    try:
        print(f" → read {i+1}/{len(input_S1_files)} scene: {os.path.basename(file_path)}")
        s1_src = esa_snappy.ProductIO.readProduct(file_path)  
        s1_src_all.append(s1_src)   # save all files into a list
        file_name.append(s1_src.getName())

        date_str = os.path.basename(file_path).split('_')[4][:8]
        file_date.append(date_str)

        print(f"  read successfully - size: {s1_src.getSceneRasterWidth()} x {s1_src.getSceneRasterHeight()}")
        
    except Exception as e:
        print(f"  read failure: {str(e)}")
        continue


In [ ]:
# Call gpt -h from command line， -h: help
print(subprocess.Popen(['gpt','-h'], stdout=subprocess.PIPE, universal_newlines=True).communicate()[0])

#set a function to get gpt help
def get_gpt_help(operator):
    result = subprocess.run(
        ['gpt', '-h', operator],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        return result.stdout
    else:
        return f"error：{result.stderr}"


In [ ]:
# get_gpt_help() testing
# print(get_gpt_help('Write'))

In [ ]:
from tqdm import tqdm

pbar = tqdm(zip(s1_src_all, file_name, file_date), total=len(s1_src_all))

for s1_src, name, date in pbar:
    # Update progress bar description: Display the name of the image currently being processed
    pbar.set_description(f"processing {name}")

# for s1_src, name, date in zip(s1_src_all, file_name, file_date):

    ### Step 1: Apply Orbit File
    # Parameter settings
    parameters1 = esa_snappy.HashMap()
    parameters1.put('orbitType', 'Sentinel Precise (Auto Download)')
    parameters1.put('continueOnFail', True)

    tem = esa_snappy.GPF.createProduct('Apply-Orbit-File', parameters1, s1_src)

    
    ### Step 2: Thermal Noise Removal
    # Parameter settings
    parameters2 = esa_snappy.HashMap()
    parameters2.put('removeThermalNoise', True)
    parameters2.put('selectedPolarisations', 'HH')

    tem = esa_snappy.GPF.createProduct('ThermalNoiseRemoval', parameters2, tem)


    ### Step 3: Radiometric Calibration
    # Parameter settings
    parameters3 = esa_snappy.HashMap()
    parameters3.put('selectedPolarisations', 'HH')
    parameters3.put('outputImageScaleInDb', False)
    parameters3.put('outputSigmaBand', True)

    tem = esa_snappy.GPF.createProduct("Calibration", parameters3, tem)

    
    ### Step 4: Speckle Flitering
    # Parameter settings
    parameters4 = esa_snappy.HashMap()
    parameters4.put('filter', 'Refined Lee')

    tem = esa_snappy.GPF.createProduct('Speckle-Filter', parameters4, tem)

    
    ### Step 5: Terrain Correction
    # Parameter settings
    proj = '''PROJCS["WGS 84 / Antarctic Polar Stereographic", 
      GEOGCS["WGS 84", 
        DATUM["World Geodetic System 1984", 
          SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], 
          AUTHORITY["EPSG","6326"]], 
        PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], 
        UNIT["degree", 0.017453292519943295], 
        AXIS["Geodetic longitude", EAST], 
        AXIS["Geodetic latitude", NORTH], 
        AUTHORITY["EPSG","4326"]], 
      PROJECTION["Polar Stereographic (variant B)", AUTHORITY["EPSG","9829"]], 
      PARAMETER["central_meridian", 0.0], 
      PARAMETER["Standard_Parallel_1", -71.0], 
      PARAMETER["false_easting", 0.0], 
      PARAMETER["false_northing", 0.0], 
      UNIT["m", 1.0], 
      AXIS["Easting", "North along 90 deg East"], 
      AXIS["Northing", "North along 0 deg"], 
      AUTHORITY["EPSG","3031"]]'''

    parameters5 = esa_snappy.HashMap()
    parameters5.put('sourceBands', 'Sigma0_HH')
    parameters5.put('demName', 'GETASSE30')
    parameters5.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters5.put('mapProjection', proj)

    tem = esa_snappy.GPF.createProduct('Terrain-Correction', parameters5, tem)

    
    ### Step 6： Convert sigma nought to dB
    # Parameter settings
    parameters6 = esa_snappy.HashMap()

    tem = esa_snappy.GPF.createProduct('LinearToFromdB', parameters6, tem)

    
    ### Step 7: Subset: zoom to the study area Atka Bay
    # Parameter settings
    wkt_polygon = "POLYGON((-8.2 -70.7, -7.4 -70.7, -7.4 -70.5, -8.2 -70.5, -8.2 -70.7))"

    parameters7 = esa_snappy.HashMap()
    parameters7.put('copyMetadata', True)
    parameters7.put('geoRegion', wkt_polygon)
    parameters7.put('fullSwath', False)

    tem = esa_snappy.GPF.createProduct('Subset', parameters7, tem)


    ### Step 8: Write the pre-processed well products for Atka Bay
    output_filename = f"S1_AtkaBay_{date}_HH_sigma0_dB.tif" 
    output_path = os.path.join(output_dir, output_filename)
    esa_snappy.ProductIO.writeProduct(tem, output_path, 'GeoTIFF')
